# Assignment 2 
### Anh Tu Vu a1911757

To view all related files used in this notebook, please refer to the full project at: https://github.com/vuanhtu52/anlp-assignment2.git

## A. Tasks as specified for your team structure

**One headings for each task.**

### Data Preprocessing

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import pipeline
import json
from tqdm import tqdm
import subprocess
from langdetect import detect
from tqdm import tqdm
tqdm.pandas()
import re
import contractions
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
import neuralcoref
import textwrap
from gensim.models import Word2Vec
import collections
import string
from tabulate import tabulate
import allennlp_models
from allennlp.predictors.predictor import Predictor

/Users/vuanhtu52/Desktop/Applied Natural Language Processing/Assignments/Assignment 2/anlp-assignment2/anlp_ass2/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/news_dataset.csv", encoding="latin-1")

In [3]:
df.head(5)

,id,author,date,year,month,topic,article
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS ? When the Islamic State was about to...
1,17292,Andy Newman,31/12/2016,2016,12,art,Angels are everywhere in the Mu?iz family?s ap...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,Finally. The Second Avenue subway opened in Ne...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON ? It?s or time for Republica...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,"For Megyn Kelly, the shift from Fox News to NB..."


In [4]:
df.isna().sum()

id         0
author     6
date       0
year       0
month      0
topic      0
article    0
dtype: int64

No missing values in our dataset

In [5]:
def is_english(text):
    return detect(text) == "en"

In [6]:
articles = df["article"].to_list()
for article in tqdm(articles):
    if not is_english(article):
        print(article)

100%|██████████| 1000/1000 [00:13<00:00, 76.23it/s]


All articles are in English, so we do not have to discard any sample.

In [7]:
# Load spacy English model
# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

def preprocess(text, lemmatize=True):
    """Preprocess the article for word2vec model"""
    text_cleaned = ""

    # Lowercase the text
    text = text.lower()

    # Remove ?
    text_cleaned = text.replace("?", "")

    # Remove extra white spaces
    text_cleaned = re.sub(" +", " ", text_cleaned).strip()

    # Tokenize the text
    tokens = word_tokenize(text_cleaned)
    text_cleaned = " ".join(tokens)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    text_cleaned = " ".join([word for word in text_cleaned.split(" ") if word not in stop_words])

    # Lemmatize
    if lemmatize:
        doc = nlp(text_cleaned)
        lemmatized_tokens = [token.lemma_ for token in doc]
        text_cleaned = ' '.join(lemmatized_tokens)

    return text_cleaned


In [8]:
# df["article_cleaned"] = df["article"].progress_apply(preprocess)
# df.to_csv("data/news_dataset_cleaned.csv", index=False)

In [9]:
df = pd.read_csv("data/news_dataset_cleaned.csv")
df.head(5)

,id,author,date,year,month,topic,article,article_cleaned
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS ? When the Islamic State was about to...,pari islamic state drive ancient city palmyra ...
1,17292,Andy Newman,31/12/2016,2016,12,art,Angels are everywhere in the Mu?iz family?s ap...,angel everywhere muiz familys apartment bronx ...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,Finally. The Second Avenue subway opened in Ne...,finally . second avenue subway open new york c...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON ? It?s or time for Republica...,washington time republican . tumultuous decade...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,"For Megyn Kelly, the shift from Fox News to NB...","megyn kelly , shift fox news nbc host daily da..."


In [10]:
# Get the test data
test_data = {}
with open("data/test_data.txt") as f:
    lines = f.readlines()
lines = [line.replace("\n", "") for line in lines]

for line in lines:
    id = line.split("|")[0]
    question = line.split("|")[1]
    answer = line.split("|")[2]
    article_id = int(line.split("|")[3])
    test_data[id] = {
        "question": question,
        "answer": answer,
        "article_id": article_id
    }

print(json.dumps(test_data, indent=4))

{
    "1": {
        "question": "Who was the President during the conflict?",
        "answer": "George W. Bush",
        "article_id": 17311
    },
    "2": {
        "question": "Who is the Senator of Colorado?",
        "answer": "Cory Gardner",
        "article_id": 17311
    },
    "3": {
        "question": "What was the revolt?",
        "answer": "Tea Party Revolt",
        "article_id": 17311
    },
    "4": {
        "question": "When did they get control back?",
        "answer": "2010",
        "article_id": 17311
    },
    "5": {
        "question": "Where is the senior Republican from?",
        "answer": "Oklahoma",
        "article_id": 17311
    },
    "6": {
        "question": "Who was the president during the Iraq War?",
        "answer": "George W. Bush",
        "article_id": 17311
    },
    "7": {
        "question": "What amount did Fox News offer?",
        "answer": "20 Million",
        "article_id": 17339
    },
    "8": {
        "question": "Where did C

### Functions used for evaluation later

In [11]:
# Taken from Evaluation Script v2.0: https://rajpurkar.github.io/SQuAD-explorer/
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

# Taken from Evaluation Script v2.0: https://rajpurkar.github.io/SQuAD-explorer/
def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

# Taken from Evaluation Script v2.0: https://rajpurkar.github.io/SQuAD-explorer/
def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def compute_exact_match(a_gold, a_pred):
  if a_gold == a_pred:
    return 1
  return 0

# Taken from this repo: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
  """
  Computes the average precision at k.

  This function computes the average prescision at k between two lists of
  items.

  Parameters
  ----------
  actual : list
            A list of elements that are to be predicted (order doesn't matter)
  predicted : list
              A list of predicted elements (order does matter)
  k : int, optional
      The maximum number of predicted elements

  Returns
  -------
  score : double
          The average precision at k over the input lists

  """
  if len(predicted)>k:
      predicted = predicted[:k]

  score = 0.0
  num_hits = 0.0

  for i,p in enumerate(predicted):
      if p in actual and p not in predicted[:i]:
          num_hits += 1.0
          score += num_hits / (i+1.0)

  if not actual:
      return 0.0

  return score / min(len(actual), k)

# Taken from this repo: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def mapk(actual, predicted, k=10):
  """
  Computes the mean average precision at k.

  This function computes the mean average prescision at k between two lists
  of lists of items.

  Parameters
  ----------
  actual : list
            A list of lists of elements that are to be predicted 
            (order doesn't matter in the lists)
  predicted : list
              A list of lists of predicted elements
              (order matters in the lists)
  k : int, optional
      The maximum number of predicted elements

  Returns
  -------
  score : double
          The mean average precision at k over the input lists

  """
  return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def mrr(actual, predicted):
  reciprocal_ranks = []

  for i in range(len(actual)):
    truth = actual[i]
    prediction = predicted[i]

    for j, sent in enumerate(prediction):
      if sent in truth:
        reciprocal_ranks.append(1 / (j+1))
        break
         
  if sum(reciprocal_ranks) == 0:
     return 0
         
  return sum(reciprocal_ranks) / len(reciprocal_ranks)


### Classic Approach

In [12]:
# Train word2vec model on all the articles
w2v_model = Word2Vec(df["article_cleaned"].apply(word_tokenize).to_list(), min_count=1)

def text_to_vector(text):
    """Convert a sentence to vector using word2vec"""
    tokens = word_tokenize(text)
    return sum(w2v_model.wv[token] for token in tokens) / len(tokens)

def cosine_similarity(v1, v2):
        # Calculate the dot product between v1 and v2
        dot_product = np.dot(v1, v2)

        # Calculate |v1| . |v2|
        magnitude = np.sqrt(np.sum(v1 ** 2) * np.sum(v2 ** 2))

        return dot_product / magnitude

In [13]:
nlp = spacy.load("en_core_web_sm")
neuralcoref.add_to_pipe(nlp)

def sort_sentences(question, article):
    """Split the article into sentences and sort them based on relevance to the question"""
    # Preprocess the article
    article = article.replace("?", "")
    article = re.sub(" +", " ", article).strip()

    # Resolve coreferences
    doc = nlp(article)
    article = nlp(doc._.coref_resolved).text

    # Split to sentences
    sentences = sent_tokenize(article)

    # Calculate cosine similarity between each sentence and question
    similarities = []
    question_vector = text_to_vector(preprocess(question))
    for sent in sentences:
        try:
            sent_vector = text_to_vector(preprocess(sent))
        except Exception:
            similarities.append(0)
            continue
        similarity = cosine_similarity(question_vector, sent_vector)
        similarities.append(similarity)

    # Save everything to a dataframe
    sent_df = pd.DataFrame()
    sent_df["id"] = [_id for _id in range(len(sentences))]
    sent_df["sentence"] = sentences
    sent_df["similarity"] = similarities

    # Sort based on similarity
    sent_df = sent_df.sort_values("similarity", ascending=False)

    return sent_df["sentence"].to_list()

def answer_question_classic(question, article):
    """Return a snippet of text from the article as answer to the question"""
    # Sort the sentences from the article based on relevance to the question
    sentences = sort_sentences(question=question, article=article)

    # Extract answer from entities based on question type
    for sent in sentences:
        doc = nlp(sent)
        for ent in doc.ents:
            if "who" in question.lower() and ent.label_ == "PERSON":
                return ent.text
            if "when" in question.lower() and ent.label_ == "DATE":
                return ent.text
            if "where" in question.lower() and ent.label_ == "LOC":
                return ent.text
            if "how many" in question.lower() and ent.label_ == "CARDINAL":
                return ent.text
            if "what amount" in question.lower() and ent.label_ == "CARDINAL":
                return ent.text
            if "what" in question.lower() and ent.label_ in ["ORG", "LOC", "NORP", "GPE"]:
                return ent.text
            
    return ""


In [14]:
# Evaluate on custom test set
f1_scores = []
em_scores = []

for _id in tqdm(test_data, desc="Evaluating"):
    question = test_data[_id]["question"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]
    answer_gold = test_data[_id]["answer"]

    answer_predicted = answer_question_classic(question=question, article=article)
    f1_scores.append(compute_f1(a_gold=answer_gold, a_pred=answer_predicted))
    em_scores.append(compute_exact_match(a_gold=answer_gold, a_pred=answer_predicted))

print(f"Average f1: {sum(f1_scores) / len(f1_scores)}")
print(f"Average exact match scores: {sum(em_scores) / len(em_scores)}")

Evaluating: 100%|██████████| 41/41 [02:34<00:00,  3.77s/it]

Average f1: 0.18292682926829268
Average exact match scores: 0.12195121951219512


In [15]:
# Evaluate how well the system finds the relevant sentences on custom test set

truths = []
predictions = []

for _id in tqdm(test_data, desc="Evaluating"):
    # Extraction information from test dataset
    question = test_data[_id]["question"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]
    answer_gold = test_data[_id]["answer"]

    # Get the ranked sentences from the system
    sentences = sort_sentences(question=question, article=article)
    prediction = sentences

    # Get the relevant sentences 
    truth = []
    for sent in sentences:
        if answer_gold.lower() in sent.lower():
            truth.append(sent) 

    truths.append(truth)
    predictions.append(prediction)  

print(tabulate([
    [
        mapk(actual=truths, predicted=predictions, k=5),
        mapk(actual=truths, predicted=predictions, k=10),
        mapk(actual=truths, predicted=predictions, k=15),
        mrr(actual=truths, predicted=predictions)
    ]
], headers=["MAP@5", "MAP@10", "MAP@15", "MRR"]))

Evaluating: 100%|██████████| 41/41 [02:28<00:00,  3.63s/it]

   MAP@5    MAP@10    MAP@15       MRR
--------  --------  --------  --------
0.143171  0.173056  0.184014  0.299168


In [22]:
# Train the word2vec model on squad train dataset

# with open("squad/train-v2.0.json", "r") as f:
#     squad_train = json.load(f)

# articles = []
# for article in squad_train["data"]:
#     for paragraph in article["paragraphs"]:
#         articles.append(paragraph["context"])

# articles = list(set(articles))
# for i, article in tqdm(enumerate(articles), total=len(articles)):
#     articles[i] = preprocess(article)

# w2v_model_2 = Word2Vec(articles, min_count=1)

100%|██████████| 19029/19029 [16:09<00:00, 19.63it/s]


In [23]:
# w2v_model_2.save("squad/models/w2v_squad.model")

In [24]:
w2v_model_2 = Word2Vec.load("squad/models/w2v_squad.model")

In [25]:
def preprocess_article(text):
    """Preprocessing the articles before feeding into the models"""
    text_cleaned = ""

    # Remove ?
    text_cleaned = text.replace("?", "")

    # Remove extra white spaces
    text_cleaned = re.sub(" +", " ", text_cleaned).strip()

    return text_cleaned

In [29]:
# Generate predictions on the squad dev set

# f1_scores = []
# em_scores = []

# with open("squad/dev-v2.0.json", "r") as f:
#     data = json.load(f)

# ids = []
# answers = []
# for article in tqdm(data["data"], desc="Articles"):
#         for paragraph in article["paragraphs"]:
#             context = paragraph["context"]
#             for qa in paragraph["qas"]:
#                 try:
#                     answer_predicted = answer_question_classic(question=qa["question"], article=preprocess_article(context))
#                     answers.append(answer_predicted)
#                     ids.append(qa["id"])
#                 except Exception:
#                      answers.append("")
#                      ids.append(qa["id"])


# predictions = {}
# for i, _ in enumerate(ids):
#     predictions[ids[i]] = answers[i]

# with open("squad/predictions/classic.json", "w") as f:
#     json.dump(predictions, f)

Articles: 100%|██████████| 35/35 [1:04:37<00:00, 110.79s/it]


In [31]:
# Evaluate on squad dev set
subprocess.run(["python", "squad/evaluate-v2.0.py"] + ["squad/dev-v2.0.json", "squad/predictions/classic.json"])

{
  "exact": 34.0604733428788,
  "f1": 34.75574425638583,
  "total": 11873,
  "HasAns_exact": 4.301619433198381,
  "HasAns_f1": 5.694155120794402,
  "HasAns_total": 5928,
  "NoAns_exact": 63.734230445752736,
  "NoAns_f1": 63.734230445752736,
  "NoAns_total": 5945
}


CompletedProcess(args=['python', 'squad/evaluate-v2.0.py', 'squad/dev-v2.0.json', 'squad/predictions/classic.json'], returncode=0)

### BiDAF

In [17]:
# Load the model
predictor = Predictor.from_path("hf://allenai/bidaf")

/Users/vuanhtu52/Desktop/Applied Natural Language Processing/Assignments/Assignment 2/anlp-assignment2/anlp_ass2/lib/python3.7/site-packages/huggingface_hub/file_download.py:654: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  FutureWarning,
Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 35246.25it/s]


In [18]:
# Evaluate on custom test set
f1_scores = []
em_scores = []

for _id in tqdm(test_data, desc="Evaluating"):
    question = test_data[_id]["question"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]
    answer_gold = test_data[_id]["answer"]

    answer_predicted = predictor.predict_json({
        "passage": preprocess_article(article),
        "question": question
    })["best_span_str"]

    f1_scores.append(compute_f1(a_gold=answer_gold, a_pred=answer_predicted))
    em_scores.append(compute_exact_match(a_gold=answer_gold, a_pred=answer_predicted))

print(f"Average f1: {sum(f1_scores) / len(f1_scores)}")
print(f"Average exact match scores: {sum(em_scores) / len(em_scores)}")

Evaluating: 100%|██████████| 41/41 [00:10<00:00,  3.85it/s]

Average f1: 0.6053832777658673
Average exact match scores: 0.36585365853658536


In [19]:
# Generate the predictions on squad dataset

# with open("squad/dev-v2.0.json", "r") as f:
#         data = json.load(f)
# ids = []
# answers = []
# for article in tqdm(data["data"], desc="Articles"):
#     for paragraph in article["paragraphs"]:
#         context = paragraph["context"]
#         for qa in paragraph["qas"]:
#             answer_predicted = predictor.predict_json({
#                  "passage": preprocess_article(context),
#                 "question": qa["question"]
#             })["best_span_str"]
#             answers.append(answer_predicted)
#             ids.append(qa["id"])

# predictions = {}
# for i, _ in enumerate(ids):
#     predictions[ids[i]] = answers[i]

# with open("squad/predictions/bidaf.json", "w") as f:
#     json.dump(predictions, f)

In [20]:
# Evaluate on squad dataset
subprocess.run(["python", "squad/evaluate-v2.0.py"] + ["squad/dev-v2.0.json", "squad/predictions/bidaf.json"])

{
  "exact": 33.95940368904237,
  "f1": 38.74344064147275,
  "total": 11873,
  "HasAns_exact": 68.01619433198381,
  "HasAns_f1": 77.59798764106039,
  "HasAns_total": 5928,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 5945
}


CompletedProcess(args=['python', 'squad/evaluate-v2.0.py', 'squad/dev-v2.0.json', 'squad/predictions/bidaf.json'], returncode=0)

### SpanBERT

In [21]:
# Load model
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/spanbert-finetuned-squadv2",
    tokenizer="mrm8488/spanbert-finetuned-squadv2"
)

In [41]:
# Evaluate on custom test set
f1_scores = []
em_scores = []

for _id in tqdm(test_data, desc="Evaluating"):
    question = test_data[_id]["question"]
    answer_gold = test_data[_id]["answer"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]

    answer_predicted = qa_pipeline({
        "context": preprocess_article(article),
        "question": question
    })["answer"]
    
    f1_scores.append(compute_f1(a_gold=answer_gold, a_pred=answer_predicted))
    em_scores.append(compute_exact_match(a_gold=answer_gold, a_pred=answer_predicted))
    
print(f"Average f1: {sum(f1_scores) / len(f1_scores)}")
print(f"Average exact match scores: {sum(em_scores) / len(em_scores)}")

Evaluating: 100%|██████████| 41/41 [00:44<00:00,  1.08s/it]

Average f1: 0.6110917537746806
Average exact match scores: 0.4634146341463415


In [23]:
# Generate the predictions on squad dataset

# with open("squad/dev-v2.0.json", "r") as f:
#         data = json.load(f)
# ids = []
# answers = []
# for article in tqdm(data["data"], desc="Articles"):
#     for paragraph in article["paragraphs"]:
#         context = paragraph["context"]
#         for qa in paragraph["qas"]:
#             answer_predicted = qa_pipeline({
#                 "context": preprocess_article(context),
#                 "question": qa["question"]
#             })["answer"]
#             answers.append(answer_predicted)
#             ids.append(qa["id"])

# predictions = {}
# for i, _ in enumerate(ids):
#     predictions[ids[i]] = answers[i]

# with open("squad/predictions/spanbert.json", "w") as f:
#     json.dump(predictions, f)

In [24]:
# Evaluate on squad dataset
subprocess.run(["python", "squad/evaluate-v2.0.py"] + ["squad/dev-v2.0.json", "squad/predictions/spanbert.json"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 41.56489514023414,
  "f1": 45.378712108463006,
  "total": 11873,
  "HasAns_exact": 83.1140350877193,
  "HasAns_f1": 90.75260608363384,
  "HasAns_total": 5928,
  "NoAns_exact": 0.1345668629100084,
  "NoAns_f1": 0.1345668629100084,
  "NoAns_total": 5945
}


CompletedProcess(args=['python', 'squad/evaluate-v2.0.py', 'squad/dev-v2.0.json', 'squad/predictions/spanbert.json'], returncode=0)

### BERT Base Model

In [25]:
model_name = "deepset/bert-base-cased-squad2"

# Load the model and tokenizer
bert_base_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_base_model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [26]:
def answer_question_bert_base(question, article):
    # Encode the question and passage using the tokenizer
    inputs = bert_base_tokenizer.encode_plus(question, article, return_tensors="pt", max_length=512, truncation=True, truncation_strategy="only_second")
    input_ids = inputs["input_ids"]
    token_type_ids = inputs["token_type_ids"]
    attention_mask = inputs["attention_mask"]

    # Pass the encoded input through the model
    outputs = bert_base_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Decode the predicted start and end positions to get the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1

    # input_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer_tokens = input_ids[0][start_index:end_index]
    answer = bert_base_tokenizer.decode(answer_tokens)

    # Skip over any tokens before the start position or after the end position
    for token in answer_tokens:
        if token == bert_base_tokenizer.cls_token_id:
            start_index += 1
        elif token == bert_base_tokenizer.sep_token_id:
            end_index -= 1
    answer_tokens = input_ids[0][start_index:end_index]

    # Decode the answer tokens to get the final answer
    answer = bert_base_tokenizer.decode(answer_tokens)

    return answer

In [27]:
# Evaluate answers on custom test set
f1_scores = []
em_scores = []

for _id in tqdm(test_data, desc="Evaluating"):
    question = test_data[_id]["question"]
    answer_gold = test_data[_id]["answer"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]

    answer_predicted = answer_question_bert_base(question=question, article=preprocess_article(article))
    f1_scores.append(compute_f1(a_gold=answer_gold, a_pred=answer_predicted))
    em_scores.append(compute_exact_match(a_gold=answer_gold, a_pred=answer_predicted))
    
print(f"Average f1: {sum(f1_scores) / len(f1_scores)}")
print(f"Average exact match scores: {sum(em_scores) / len(em_scores)}")

Evaluating: 100%|██████████| 41/41 [00:13<00:00,  2.96it/s]

Average f1: 0.220709262307347
Average exact match scores: 0.1951219512195122


In [28]:
# Generate predictions on squad dataset

# with open("squad/dev-v2.0.json", "r") as f:
#         data = json.load(f)
# ids = []
# answers = []
# for article in tqdm(data["data"], desc="Articles"):
#     for paragraph in article["paragraphs"]:
#         context = paragraph["context"]
#         for qa in paragraph["qas"]:
#             answer_predicted = answer_question_bert_base(question=qa["question"], article=preprocess_article(context))
#             answers.append(answer_predicted)
#             ids.append(qa["id"])

# predictions = {}
# for i, _ in enumerate(ids):
#     predictions[ids[i]] = answers[i]

# with open("squad/predictions/bert_base.json", "w") as f:
#     json.dump(predictions, f)

Articles: 100%|██████████| 35/35 [26:46<00:00, 45.90s/it]


In [29]:
# Evaluate on squad dataset
subprocess.run(["python", "squad/evaluate-v2.0.py"] + ["squad/dev-v2.0.json", "squad/predictions/bert_base.json"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 67.21131980122968,
  "f1": 71.39165741443425,
  "total": 11873,
  "HasAns_exact": 58.232118758434545,
  "HasAns_f1": 66.60478213251974,
  "HasAns_total": 5928,
  "NoAns_exact": 76.16484440706476,
  "NoAns_f1": 76.16484440706476,
  "NoAns_total": 5945
}


CompletedProcess(args=['python', 'squad/evaluate-v2.0.py', 'squad/dev-v2.0.json', 'squad/predictions/bert_base.json'], returncode=0)

### DISTILBERT Model

In [30]:
# Load the model and tokenizer
model_name = "distilbert-base-cased-distilled-squad"
distilbert_tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [31]:
def answer_question_distilbert(question, article):
    # Encode the question and passage using the tokenizer
    inputs = distilbert_tokenizer.encode_plus(question, article, return_tensors="pt", max_length=512, truncation=True, truncation_strategy="only_second")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Pass the encoded input through the QA model
    outputs = distilbert_model(input_ids, attention_mask, return_dict=True)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Decode the predicted start and end positions to get the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1

    # Skip over any tokens before the start position or after the end position
    for j, token_id in enumerate(input_ids[0]):
        if j < start_index or j >= end_index:
            input_ids[0][j] = distilbert_tokenizer.pad_token_id

    # Decode the answer from the corresponding tokens
    answer_tokens = input_ids[0][start_index:end_index]
    answer = distilbert_tokenizer.decode(answer_tokens)

    return answer

In [32]:
# Evaluate on test set
f1_scores = []
em_scores = []

for _id in tqdm(test_data, desc="Evaluating"):
    question = test_data[_id]["question"]
    answer_gold = test_data[_id]["answer"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]

    answer_predicted = answer_question_distilbert(question=question, article=preprocess_article(article))
    f1_scores.append(compute_f1(a_gold=answer_gold, a_pred=answer_predicted))
    em_scores.append(compute_exact_match(a_gold=answer_gold, a_pred=answer_predicted))
    
print(f"Average f1: {sum(f1_scores) / len(f1_scores)}")
print(f"Average exact match scores: {sum(em_scores) / len(em_scores)}")

Evaluating: 100%|██████████| 41/41 [00:07<00:00,  5.58it/s]

Average f1: 0.3310104529616725
Average exact match scores: 0.24390243902439024


In [33]:
# Generate predictions on squad dataset

# with open("squad/dev-v2.0.json", "r") as f:
#         data = json.load(f)
# ids = []
# answers = []
# for article in tqdm(data["data"], desc="Articles"):
#     for paragraph in article["paragraphs"]:
#         context = paragraph["context"]
#         for qa in paragraph["qas"]:
#             answer_predicted = answer_question_distilbert(question=qa["question"], article=preprocess_article(context))
#             answers.append(answer_predicted)
#             ids.append(qa["id"])

# predictions = {}
# for i, _ in enumerate(ids):
#     predictions[ids[i]] = answers[i]

# with open("squad/predictions/distilbert.json", "w") as f:
#     json.dump(predictions, f)

In [34]:
# Evaluate on squad dataset
subprocess.run(["python", "squad/evaluate-v2.0.py"] + ["squad/dev-v2.0.json", "squad/predictions/distilbert.json"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 36.64617198686094,
  "f1": 41.272836055899646,
  "total": 11873,
  "HasAns_exact": 71.17071524966262,
  "HasAns_f1": 80.43731148645352,
  "HasAns_total": 5928,
  "NoAns_exact": 2.220353238015139,
  "NoAns_f1": 2.220353238015139,
  "NoAns_total": 5945
}


CompletedProcess(args=['python', 'squad/evaluate-v2.0.py', 'squad/dev-v2.0.json', 'squad/predictions/distilbert.json'], returncode=0)

### BERT Large

In [35]:
# Load model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
bert_large_model = BertForQuestionAnswering.from_pretrained(model_name)
bert_large_tokenizer = BertTokenizer.from_pretrained(model_name)

In [36]:
def answer_question_bert_large(question, article):
    # ======== Tokenize ========
    # Apply the tokenizer to the encode text, treating them as a question, answer_text pair.
    input_ids = bert_large_tokenizer.encode(question, article, max_length=512)

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(bert_large_tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    model_scores = bert_large_model.forward(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text
    
    start_scores = model_scores.start_logits
    end_scores = model_scores.end_logits
    
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = bert_large_tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        # If it's a subword token, then recombine it with the previous token.
        # print(tokens[i])
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    return answer

In [37]:
# Evaluate on test set

import logging
logging.disable(logging.WARNING)

f1_scores = []
em_scores = []

for _id in tqdm(test_data, desc="Evaluating"):
    question = test_data[_id]["question"]
    answer_gold = test_data[_id]["answer"]
    article = df.loc[df["id"] == test_data[_id]["article_id"], "article"].values[0]

    answer_predicted = answer_question_bert_large(question=question, article=preprocess_article(article))
    f1_scores.append(compute_f1(a_gold=answer_gold, a_pred=answer_predicted))
    em_scores.append(compute_exact_match(a_gold=answer_gold, a_pred=answer_predicted))
    
print(f"Average f1: {sum(f1_scores) / len(f1_scores)}")
print(f"Average exact match scores: {sum(em_scores) / len(em_scores)}")

Evaluating: 100%|██████████| 41/41 [00:44<00:00,  1.09s/it]

Average f1: 0.433925271882589
Average exact match scores: 0.04878048780487805


In [38]:
# Generate predictions on squad dataset

# with open("squad/dev-v2.0.json", "r") as f:
#         data = json.load(f)
# ids = []
# answers = []
# for article in tqdm(data["data"], desc="Articles"):
#     for paragraph in article["paragraphs"]:
#         context = paragraph["context"]
#         for qa in paragraph["qas"]:
#             answer_predicted = answer_question_bert_large(question=qa["question"], article=preprocess_article(context))
#             answers.append(answer_predicted)
#             ids.append(qa["id"])

# predictions = {}
# for i, _ in enumerate(ids):
#     predictions[ids[i]] = answers[i]

# with open("squad/predictions/bert_large.json", "w") as f:
#     json.dump(predictions, f)

In [39]:
# Evaluate on squad dataset
subprocess.run(["python", "squad/evaluate-v2.0.py"] + ["squad/dev-v2.0.json", "squad/predictions/bert_large.json"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 37.993767371346756,
  "f1": 42.75583509830704,
  "total": 11873,
  "HasAns_exact": 75.86032388663968,
  "HasAns_f1": 85.39811574261124,
  "HasAns_total": 5928,
  "NoAns_exact": 0.23549201009251472,
  "NoAns_f1": 0.23549201009251472,
  "NoAns_total": 5945
}


CompletedProcess(args=['python', 'squad/evaluate-v2.0.py', 'squad/dev-v2.0.json', 'squad/predictions/bert_large.json'], returncode=0)

### User Interaction System

In [40]:
def start_app():
    while True:
        # Ask user to select a model
        print("Please select a model to answer your question: \n1. Classic model\n2. BiDAF\n3. SpanBERT\n4. BERT base\n5. DISTILBERT\n6.BERT large")
        model_id = ""
        while True:
            model_id = input("Enter model id: ")

            if model_id == "quit":
                break

            try:
                model_id = int(model_id)
            except Exception:
                print("Wrong id. Enter again: ")
                continue

            if int(model_id) >= 1 and int(model_id) <= 6:
                break
            else:
                print("Wrong id. Enter again: ")

        if model_id == "quit":
            break

        print(f"Model selected: {model_id}\n")


        # Ask user to enter article number
        # article_id = input("Please select an article id: ")
        article_id = ""
        article = ""
        while True:
            article_id = input("Please select an article id: ")

            if article_id == "quit":
                break

            try:
                article_id = int(article_id)
                article = df.loc[df["id"] == article_id, "article"].values[0]
                break
            except Exception:
                print("Wrong id. Enter again: ")
                continue
        
        if article_id == "quit":
            break
        
        print(f"Article id selected: {article_id}")

        # Ask user to enter the question
        question = input("Please enter your question: ")
        if question == "quit":
            break
        print("Question: ", question)

        # Generate answer
        answer = ""
        if model_id == 1:
            answer = answer_question_classic(question=question, article=article)
        elif model_id == 2:
            answer = predictor.predict_json({
                        "passage": preprocess_article(article),
                        "question": question
                    })["best_span_str"]
        elif model_id == 3:
            answer = qa_pipeline({
                        "context": preprocess_article(article),
                        "question": question
                    })["answer"]
        elif model_id == 4:
            answer = answer_question_bert_base(question=question, article=preprocess_article(article))
        elif model_id == 5:
            answer = answer_question_distilbert(question=question, article=preprocess_article(article))
        elif model_id == 6:
            answer = answer_question_bert_large(question=question, article=preprocess_article(article))

        if answer == "":
            answer = "No answer found"

        print(f"Answer: {answer}")
        print()

# Uncomment this line to run the app
# start_app()

## B. References

## C. Appendix